Run the cell below if you are using Google Colab to mount your Google Drive in your Colab instance. Adjust the path to the files in your Google Drive as needed if it differs.

If you do not use Google Colab, running the cell will simply do nothing, so do not worry about it.

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    %cd 'drive/My Drive/Colab Notebooks/06_Classification'
except ImportError as e:
    pass

## Exercise 6: Neural Networks

### 6.1. Learning a neural net with scikit-learn for the Adult dataset

In this task, you should create a neural network for the Adult dataset. Use the MLPClassifier provided by scikit-learn.

#### 6.1.1	Load the Adults dataset and play around with the hidden layers and the activation function (both are architecture parameters)

In [ ]:
import pandas as pd
import numpy as np
from scipy.io import arff
from sklearn.preprocessing import LabelEncoder

adult_arff_data, adult_arff_meta = arff.loadarff(open('adult.arff', 'r'))
adult = pd.DataFrame(adult_arff_data)
adult = adult.applymap(lambda x: x.decode('utf8').replace("'", "") if hasattr(x, 'decode') else x)

adult_target = adult['class']
label_encoder = LabelEncoder()
adult_target = label_encoder.fit_transform(adult_target)
adult_data = adult.drop('class', axis=1)
adult_data.head()

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(sparse=False, handle_unknown='ignore'), categorical_features)])

In [ ]:
# TODO Train Test Split

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline

for hidden_layers in [#TODO: define different hidden layers]:
    print("=======" + str(hidden_layers) + "=======")
    clf = MLPClassifier(hidden_layer_sizes=hidden_layers, activation='relu',
                        learning_rate_init=1e-3, batch_size=128, verbose=True,
                        early_stopping=True, random_state=1234)

    pipeline = Pipeline([ ('preprocessing', preprocessor), ('estimator', clf) ])
    pipeline.fit(feature_train, target_train)

#### 6.1.2	Play around with the hyperparameters of the neural network like learning rate and batch size

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline

for learning_rate in [#TODO: define different learning rates]:
    print("=========== lr: " + str(learning_rate) + "===========")
    clf = MLPClassifier(hidden_layer_sizes=(100,), activation='relu',
                        learning_rate_init=learning_rate, batch_size=128, verbose=True,
                        early_stopping=True, random_state=1234)

    pipeline = Pipeline([ ('preprocessing', preprocessor), ('estimator', clf) ])

    pipeline.fit(feature_train, target_train)

In [ ]:

for batch_size in [#TODO: define different batch sizes]:
    print("=========== batch_size: " + str(batch_size) + " ===========")
    clf = MLPClassifier(hidden_layer_sizes=(100,), activation='relu',
                        learning_rate_init=1e-3, batch_size=batch_size, verbose=True,
                        early_stopping=True, random_state=1234)

    pipeline = Pipeline([ ('preprocessing', preprocessor), ('estimator', clf) ])

    pipeline.fit(feature_train, target_train)

### 6.2. Neural networks with PyTorch Lightning

In the previous exercise only the MLPClassifier is used. To extend the possibilities, use PyTorch Lightning and create a similar neural network.

#### 6.1.1	Implement the same model in PyTorch Lightning and find an optimal model using F1 as the scoring measure.

In [ ]:
# TODO: make train validation split

In [ ]:
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

# TODO: preprocess data, create TensorDataset and dataloaders

In [ ]:
import pytorch_lightning as pl
import torch.nn as nn
from torch.nn import functional as F
import torchmetrics

class LightningModel(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        self.my_model = #TODO: define model
        # TODO define metrics

    def forward(self, x):
        logits = self.my_model(x)
        return logits

    def make_step(self, x,y):
        logits = self.forward(x)
        loss = F.binary_cross_entropy_with_logits(logits, y.unsqueeze(-1))
        predictions = torch.sigmoid(logits).squeeze()
        return loss, predictions
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        loss, predictions = self.make_step(x,y)

        #TODO: call metric
        
        return {'loss' : loss}
    
    def training_epoch_end(self, training_step_outputs):
        avg_train_loss = torch.stack([x['loss'] for x in training_step_outputs]).mean()
        
        #TODO: compute epoch train f1 and print training loss 
        
    def validation_step(self, batch, batch_idx):
        x, y = batch
        loss, predictions = self.make_step(x,y)
        
        #TODO: call metric
        return {'val_loss' : loss}
    
    def validation_epoch_end(self, validation_step_outputs):
        avg_val_loss = torch.stack([x['val_loss'] for x in validation_step_outputs]).mean()
        
        #TODO: compute epoch validation f1 and print it

    def test_step(self, batch, batch_idx):
        x, y = batch
        loss, predictions = self.make_step(x,y)
        #TODO: call metric
        
        return test_f1_batch
    
    def test_epoch_end(self, test_step_outputs):
        #TODO: log metric  
        return avg_test_f1
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-4)
        return optimizer

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# set a seed for all libraries such as random, numpy random etc
pl.seed_everything(42, workers=True)
model = LightningModel()

# TODO: use logged metric
my_mectic = ''
checkpoint_callback = ModelCheckpoint(dirpath='./checkpoints', monitor=my_mectic)
early_stop_callback = EarlyStopping(monitor=my_mectic, patience=10, mode="max")
trainer = pl.Trainer(deterministic=True, 
                     max_epochs=25,
                     callbacks=[checkpoint_callback, early_stop_callback])

trainer.fit(model, train_loader, validation_loader)

In [ ]:
trainer.test(model, test_loader, ckpt_path='best')

## 6.3. Multi class classification with PyTorch Lightning
Use the Connect 4 dataset to adjust the neural network for multi class classification
The target variable to predict is either ‘win’, ‘ loss’, or ‘ draw’.

#### 6.3.1 Load the dataset and split it into training, validation and test.

In [ ]:
# TODO : load dataset
# TODO : make train validation test split
# TODO : preprocess data, create TensorDataset and dataloaders

#### 6.3.2 Adapt the architecture (change the loss and adapt the output layer) and optimize for validation accuracy.

In [ ]:
class ConnectFourModel(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        self.my_model = # TODO define model
        self.train_acc = torchmetrics.Accuracy()
        self.valid_acc = torchmetrics.Accuracy()
        self.test_acc = torchmetrics.Accuracy()

    def forward(self, x):
        logits = self.my_model(x)
        return logits
    
    def make_step(self, x, y):
        logits = self.forward(x)
        
        # TODO: calculate loss and predictions
        # loss = 
        # predictions = 
        return logits, loss, predictions
        
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits, loss, predictions = self.make_step(x,y)
        self.train_acc(predictions, y)
        return {'loss' : loss}
    
    def training_epoch_end(self, training_step_outputs):
        avg_train_loss = torch.stack([x['loss'] for x in training_step_outputs]).mean()
        avg_train_acc = self.train_acc.compute() 
        
        self.log("loss/train", avg_train_loss)
        self.log("acc/train", avg_train_acc)
        
        print('training loss at epoch ' + str(self.current_epoch) + ': ' + str(avg_train_loss.item()))
        
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits, loss, predictions = self.make_step(x, y)
        
        self.valid_acc(predictions, y)
        return {'val_loss' : loss} 
    
    def validation_epoch_end(self, validation_step_outputs):
        avg_val_loss = torch.stack([x['val_loss'] for x in validation_step_outputs]).mean()
        avg_val_acc = self.valid_acc.compute()
        
        self.log("loss/validation", avg_val_loss)
        self.log("acc/validation", avg_val_acc)
        
        print('validation accuracy at epoch ' + str(self.current_epoch) + ': ' + str(avg_val_acc.item()))

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits, loss, predictions = self.make_step(x, y)
        test_acc_batch = self.test_acc(predictions, y)
        return test_acc_batch
    
    def test_epoch_end(self, test_step_outputs):
        avg_test_acc = self.test_acc.compute()        
        self.log('acc/test', avg_test_acc)        
        return avg_test_acc

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-4)
        return optimizer

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# set a seed for all libraries such as random, numpy random etc
pl.seed_everything(42, workers=True)
model = ConnectFourModel()


checkpoint_callback = ModelCheckpoint(dirpath='./checkpoints', monitor="acc/validation")
early_stop_callback = EarlyStopping(monitor="acc/validation", patience=10, mode="max")

trainer = pl.Trainer(deterministic=True, 
                     max_epochs=25,
                     callbacks=[checkpoint_callback, early_stop_callback])

trainer.fit(model, train_loader, validation_loader)

In [ ]:
trainer.test(model, test_loader, ckpt_path='best')